# Core Model Guide: NonlinearTank

This notebook provides a detailed guide to the `NonlinearTank` model. This model is an advancement over the `LinearTank` and is used for reservoirs or tanks where the surface area changes with water level. This is typical for real-world reservoirs with sloped banks.

## 1. Theoretical Background

Like the `LinearTank`, this model is based on mass conservation. However, instead of assuming a constant surface area, it uses a user-provided **level-volume curve** to define the relationship between water level and storage volume.

The simulation logic is:
1. At the start, use the `initial_level` to look up the `initial_volume` from the curve.
2. At each time step, update the volume first: `new_volume = old_volume + (Inflow - Outflow) * dt`.
3. Use the `new_volume` to look up the `new_level` from the curve.

The model uses linear interpolation (`numpy.interp`) to find values between the points defined in the curve.

## 2. API and Parameters

Let's import the necessary libraries and the model.

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np

# Add the project root to the path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from water_system_sdk.src.chs_sdk.modules.modeling.storage_models import NonlinearTank

The `NonlinearTank` is initialized with:

- `level_to_volume` (np.ndarray): This is the key parameter. It must be a 2xN NumPy array where:
  - `array[0]` is a list of water levels.
  - `array[1]` is a corresponding list of storage volumes.
- `initial_level` (float): The water level at the beginning of the simulation.
- `min_level` (float, optional): The minimum allowed water level.
- `max_level` (float, optional): The maximum allowed water level.

## 3. Code Example

Let's simulate a V-shaped reservoir. The volume will increase with the cube of the level. We will define a simple 3-point curve to approximate this.

In [ ]:
# 1. Define the Level-Volume Curve
# Represents a reservoir that gets wider as it gets deeper
level_volume_curve = np.array([
    [0.0,   10.0,    20.0],  # levels (m)
    [0.0, 10000.0, 80000.0]  # volumes (m^3)
])

# 2. Model Initialization
tank = NonlinearTank(
    level_to_volume=level_volume_curve,
    initial_level=5.0 # Start halfway to the first data point
)

# 3. Simulation Setup
dt = 3600  # 1 hour
n_steps = 100

history = {
    "time": [],
    "level": [],
    "volume": []
}

# 4. Simulation Loop
for t in range(n_steps):
    # Constant inflow
    inflow = 10.0 # m^3/s
    
    # Set inputs
    tank.input.inflow = inflow
    tank.input.release_outflow = 0.0
    
    # Run model step
    tank.step(dt=dt)
    
    # Record results
    state = tank.get_state()
    history["time"].append(t)
    history["level"].append(state["level"])
    history["volume"].append(state["volume"])

print("Simulation complete.")

## 4. Visualization

Let's plot both the water level and the volume to see the nonlinear relationship.

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot Level
color = 'tab:blue'
ax1.set_xlabel('Time (hours)')
ax1.set_ylabel('Water Level (m)', color=color)
ax1.plot(history['time'], history['level'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

# Plot Volume on a second y-axis
ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Storage Volume (m³)', color=color)
ax2.plot(history['time'], history['volume'], color=color, linestyle='--')
ax2.tick_params(axis='y', labelcolor=color)

plt.title('NonlinearTank Simulation')
fig.tight_layout()
plt.show()

The plot shows that as the tank fills, the rate of change of the water level (the slope of the blue line) decreases over time, even though the inflow (and thus the rate of volume change) is constant. This is because at higher levels, a larger volume of water is required to raise the level by one meter, which accurately reflects the behavior of a real-world reservoir.